In [2]:
import pandas as pd
import numpy as np
# df = pd.read_csv(r'C:\Users\haech\Insta_profiling\spider\insta_crawling\gw_region.csv')

In [16]:
len(np.unique(df['inner_id']))

3719

In [18]:
# gw_region을 기반으로 id : next_page 사전화
df_dropna = df.dropna(subset=['next_page']) # next_page칼럼에 결측치가 있으면 해당 행을 삭제

In [63]:
id_ls = list(map(str, df_dropna['inner_id']))
next_ls = list(map(str, df_dropna['next_page']))

In [ ]:
# 순서를 유지하며 unique값 뽑아내기
# np.unique(id_ls)
index = np.unique(id_ls, return_index=True)[1]
uni_id_ls = [id_ls[idx] for idx in sorted(index)]

In [ ]:
index = np.unique(next_ls, return_index=True)[1]
uni_next_ls = [next_ls[idx] for idx in sorted(index)]

In [66]:
# 딕셔너리화
id2npage = {}
for i, id in enumerate(uni_id_ls):
    for idx, n_page in enumerate(uni_next_ls):
        if i == idx:
            id2npage[id] = n_page

# 한글 텍스트인 계정만 추출하기

In [25]:

import re
import pandas as pd
import os

df = pd.read_csv(r'C:\Users\haech\Insta_profiling\spider\insta_crawling\gw_region.csv')

In [26]:
id_lst = list(set(df['insta_id']))
len(id_lst)

3719

In [27]:
def del_foreign(df):
    id_lst = list(set(df['insta_id']))
    for user_id in id_lst:
        count = 0
        for y in list(df[df['insta_id']==user_id]['content']):
            if type(y) != str:
                pass
            elif re.search('[가-힣]', y):
                count += 1
        if count == 0:
            idx = df[df['insta_id']==user_id].index
            df.drop(idx, inplace=True)
    return df

In [28]:
new_df = del_foreign(df)

In [29]:
id_lst = list(set(new_df['insta_id']))
len(id_lst)

2749

In [32]:
new_df.to_csv('gw_hangul.csv', index=False)

In [50]:
gw_hangul = pd.read_csv(r'C:\Users\haech\Insta_profiling\spider\insta_crawling\gw_hangul.csv')

In [52]:
hangul_ls = list(map(str, list(np.unique(gw_hangul['inner_id']))))

In [89]:
len(hangul_ls)

2749

# 순서를 유지하며 unique값 뽑아내고, 딕셔너리 생성하기

In [85]:
def make_dic(df):
    inner_id_ls = list(map(str, list(np.unique(df['inner_id']))))
    df_dropnana = df.dropna(subset=['next_page'])
    id_ls = list(map(str, df_dropna['inner_id']))
    next_ls = list(map(str, df_dropna['next_page']))

    # 순서를 유지하며 unique값 뽑아내기
    index = np.unique(id_ls, return_index=True)[1]
    uni_id_ls = [id_ls[idx] for idx in sorted(index)]

    index = np.unique(next_ls, return_index=True)[1]
    uni_next_ls = [next_ls[idx] for idx in sorted(index)]

    # 딕셔너리 생성
    for i, id in enumerate(uni_id_ls):
        for idx, n_page in enumerate(uni_next_ls):
            if i == idx:
                id2npage[id] = n_page

In [86]:
make_dic(gw_hangul)

In [88]:
len(id2npage)

2541

# 팔로워 1000명 이상 id 제외 / 좋아요 1000개 이상 게시물 있는 id 제외

In [36]:
over1000_df = pd.read_csv(r'C:\Users\haech\Insta_profiling\spider\insta_crawling\gw_over1000.csv')

In [37]:
def del_over1000(df, over1000_df):
    # 팔로워 1000명 이상 id 제외
    for over1000_id in list(over1000_df['data']):
        idx = df[df['insta_id']==over1000_id].index
        df.drop(idx, inplace=True)
    
    # 좋아요 1000개 이상 게시물 있는 id 제외
    df['like_count'] = df['like_count'].apply(int)
    id_lst = list(set(df['insta_id']))
    for user_id in id_lst:
        count = 0
        for y in list(df[df['insta_id']==user_id]['like_count']):
            if y > 1000:
                count += 1
        if count > 1:
            idx = df[df['insta_id']==user_id].index
            df.drop(idx, inplace=True)
            
    return df

In [38]:
new_new_df = del_over1000(new_df, over1000_df)

In [40]:
id_lst = list(set(new_new_df['insta_id']))
len(id_lst)

2376

In [46]:
# new_new_df.to_csv('gw_delete.csv', index=False)

In [93]:
gw_delete = pd.read_csv(r'C:\Users\haech\Desktop\insta_project_data\gw_delete.csv')

In [103]:
len(list(map(str, list(np.unique(gw_delete['inner_id'])))))

2376

# 광고성 단어가 포함된 id제거

In [105]:
# 지역_delete 지역 경로 설정
# gw_delete_fin = pd.read_csv(r'C:\Users\haech\Insta_profiling\function\gw_delete_over_1000.csv')

In [106]:
len(list(map(str, list(np.unique(gw_delete_fin['inner_id'])))))

1928

In [73]:
# df = pd.read_csv(r'C:\Users\haech\Desktop\insta_project_data\gw_delete_over_1000.csv')

In [75]:
limit_words = ['문의', 'DM', '고객님', '완판', '이벤트', '콜라보', '납품', '광고', '구매', '매진', '할인', '홍보', '협찬', '수입', '행사', '출시', '런칭', '론칭', '입양', '한정기획', '원플원', '스마트스토어', '발매', '품절', '체험단', '운영중', '오픈시간', '티져영상', '예약', '공식쇼핑몰', '본점', '캠페인', '인스타친구', '사업', '협회', '재입고', '최대지원금', '최저가', '요금', '지급', '개시', '입점']

In [76]:
idx = []
id_list = []
word_cnt = 1 # 광고 단어 포함 횟수

for i, content in enumerate(df['content']):
    cnt = 0
    for word in limit_words:
        if word in str(content):
            cnt += 1
            if cnt == word_cnt:
                id_list.append(df['inner_id'][i])
                idx.append(i)
                break

In [77]:
for user_id in id_list:
    df = df[df['inner_id']!=user_id]
df.reset_index(inplace=True, drop=True)

In [78]:
print('제거된 포스트 개수: {}, 제거된 유저 명수: {}'.format(len(idx), len(list(set(id_list)))))

제거된 포스트 개수: 1144, 제거된 유저 명수: 398


In [80]:
df
len(list(map(str, list(np.unique(df['inner_id'])))))

1530

In [85]:
df.to_csv('gw_word.csv', index=False)

In [88]:
df1 = pd.read_csv('gw_word.csv', index_col =[0])

In [89]:
df1.to_csv('gw_word.csv', index=False)

In [3]:
dfdf = pd.read_csv('gw_word.csv')